# Training a Neural ODE

From [this blog post](https://julialang.org/blog/2019/01/fluxdiffeq/).

## Generating a toy dataset

The data were sampled in the [Lotka-Volterra equations](https://en.wikipedia.org/wiki/Lotka%E2%80%93Volterra_equations) describing the dynamics of the population of rabbits and wolves. The [Neural ODEs](https://arxiv.org/abs/1806.07366) will be trained on those samples.

In [ ]:
using DifferentialEquations

function lotka_volterra!(du,u,p,t)
  x, y = u
  α, β, δ, γ = p
  du[1] = dx = α*x - β*x*y
  du[2] = dy = -δ*y + γ*x*y
end

u0 = [1.0, 1.0]
tspan = (0.0, 10.0)
p = [1.5, 1.0, 3.0, 1.0]
prob = ODEProblem(lotka_volterra!,u0,tspan,p)

The solution looks like this:

In [ ]:
sol = solve(prob)
using Plots
plot(sol)
scatter!(t->sol(t)[1], 0:0.1:10.0, label="Data Points")

Define a neural network with the function as the single layer.

In [ ]:
using Flux, DiffEqFlux

# Initial Parameter Vector
p = [2.2, 1.0, 2.0, 0.4]
# Put the Parameter Vector into Flux.jl
params = Flux.params(p)

# The 1-layer "neural network"
function predict_rd() 
    solve(prob,Tsit5(),p=p,saveat=0.1)[1,:] # override with new parameters
end

# loss function
loss_rd() = sum(abs2, x-1 for x in predict_rd())

In [ ]:
data = Iterators.repeated((), 100)
opt = ADAM(0.1)

# callback function to observe training
cb = function () 
  display(loss_rd())
  # using `remake` to re-create our `prob` with current parameters `p`
  display(plot(solve(remake(prob,p=p),Tsit5(),saveat=0.1),ylim=(0,7)))
end

# Display the ODE with the initial parameter values.
cb()

In [ ]:
Flux.train!(loss_rd, params, data, opt, cb = cb)